In [ ]:
from graphviz import Digraph
import requests
from bs4 import BeautifulSoup
import string

In [ ]:
styles = {
    'graph': {
        'label': '',
        'fontsize': '16',
        'fontcolor': 'white',
        'bgcolor': '#333333',
        'rankdir': 'BT',
    },
    'nodes': {
        'fontname': 'Helvetica',
        'shape': 'rectangle',
        'fontcolor': 'white',
        'color': 'white',
        'style': 'filled',
        'fillcolor': '#006699',
    },
    'edges': {
        'style': 'dashed',
        'color': 'white',
        'arrowhead': 'open',
        'fontname': 'Courier',
        'fontsize': '12',
        'fontcolor': 'white',
    }
}
def apply_styles(graph, styles):
    graph.graph_attr.update(
        ('graph' in styles and styles['graph']) or {}
    )
    graph.node_attr.update(
        ('nodes' in styles and styles['nodes']) or {}
    )
    graph.edge_attr.update(
        ('edges' in styles and styles['edges']) or {}
    )
    return graph

In [ ]:
req = requests.get('http://localhost/Quest/')
html = req.text
header = req.headers
status = req.status_code
is_ok = req.ok

In [ ]:
print(status, is_ok)

In [ ]:
soup = BeautifulSoup(req.content,'html.parser', from_encoding='utf-8')


In [ ]:
my_titles = soup.select('table > tbody > tr > td')

In [ ]:
questDict = {}
for i in range(0,len(my_titles),2):
    questDict[my_titles[i].text]=my_titles[i+1].text

In [ ]:
tmp=soup.find_all('a')
questGraph=[[] for i in range(550000)]
for linki in range(0,len(tmp)):
    link=tmp[linki]
    req2=requests.get('http://localhost/Quest/'+link.get('href'))
    soup2 = BeautifulSoup(req2.content,'html.parser', from_encoding='utf-8')
    my_titles2 = soup2.select('table > tbody > tr > td > a')
    BF = soup2.select('table > tbody > tr > td')
    
    for titles in my_titles2:
        questGraph[int(titles.get('href').replace('.html',''))].append(int(link.get('href').replace('.html','')))

In [ ]:
questChk=[0 for i in range(550000)]
def dfs(x, num):
    questChk[x]=num
    for i in range(0,len(questGraph[x])):
        if questGraph[x][i] and questChk[questGraph[x][i]]==0:
            dfs(questGraph[x][i],num)
cnt=0;
for i in range(0,550000):
    if questChk[i]==0:
        cnt=cnt+1
        questChk[i]=cnt
        for j in range(0,len(questGraph[i])):
            if questGraph[i][j] and questChk[questGraph[i][j]]==0:
                dfs(questGraph[i][j],cnt)
print(cnt)


In [ ]:
def linkedGraph(index):
    questData=[]
    if questChk[index]!=0:
        try:
            for i in range(0,550000):
                if questChk[i] == questChk[index] :
                    #print(i)
                    questData.append(i);
                    dot.node(str(i), questDict[str(i)], URL="https://kennysoft.kr/WzComparerR2/Quest/"+str(i)+".html", target="_blank")
            return(questData)
        except:
            return []

In [ ]:
dot = Digraph(comment='', format='svg')
questData = linkedGraph(34459)

for data in questData:
    for graph in questGraph[data]:
        dot.edge(str(data), str(graph))

In [ ]:
print(dot.source)  # doctest: +NORMALIZE_WHITESPACE
dot = apply_styles(dot, styles)
dot.render('test-output/round-table.gv', view=True)